In [ ]:
#get the probabilities

In [1]:
import pandas as pd
import os
import re
from nltk.stem.porter import *
import math

In [2]:
data = pd.read_csv("Review_Subset.csv")

In [3]:
def getNgrams(text, n):
    if n == 1:
        return text
    
    text.append("<end>")
    ngramList = []
    for i in range(len(text) - (n-1)):
        ngram = text[i:i+n]
        ngramList.append(ngram)
    
    return ngramList

In [4]:
unigramDict = {}
bigramDict = {}
stemmer = PorterStemmer()

def getCountsinRow(row):
    text = row["text"]
    parsedText = re.findall("(\w+)", text)
    
    for index in range(len(parsedText)):
        parsedText[index] = stemmer.stem(parsedText[index])
        parsedText[index] = parsedText[index].lower()
        
    unigram = getNgrams(parsedText, 1)
    bigram = getNgrams(parsedText, 2)
    ngrams = (unigram,bigram)
    for index in range(len(ngrams)):
        for eachWord in ngrams[index]:
            
            if index == 0:
                #unigram
                
                if eachWord in unigramDict:
                    unigramDict[eachWord] +=1
                else:
                    unigramDict[eachWord] =1
                    
            if index == 1:
                #bigram
                    eachWord = tuple(eachWord)
                    if eachWord in bigramDict:
                        bigramDict[eachWord] +=1
                    else:
                        bigramDict[eachWord] =1

            

In [ ]:

#get the probabilies into unigramsDict and bigramsDict
data.apply(getCountsinRow, axis = 1)

In [ ]:
print(len(unigramDict))
print(len(bigramDict))

# SAVE UNIGRAM/BIGRAM FREQ. INTO A FILE

In [ ]:
'''
extractList = []
for eachKey in unigramDict:
    extractList.append([eachKey,unigramDict[eachKey]])

pd.DataFrame(extractList, columns = ["word", "count"]).to_csv("unigramProbabilities.csv", index = False)
'''#export unigrams


In [ ]:
#export bigrams
'''
extractList = []
for eachKey in bigramDict:
    extractList.append([eachKey[0],eachKey[1],bigramDict[eachKey]])
    
pd.DataFrame(extractList, columns = ["first", "second", "count"]).to_csv("bigramProbabilities.csv", index = False)
'''

In [ ]:
print(data)

# UNIGRAM VECTOR MODEL

In [ ]:
#get the vector model in data
unigramVM = []
def getUnigramReviewVectors(row):
    
    #we have data,unigrams,bigrams
    rowVector = {}
    text = row["text"]
    score = row["score"]
    parsedText = re.findall("(\w+)", text)
    for index in range(len(parsedText)):
        parsedText[index] = stemmer.stem(parsedText[index])
        parsedText[index] = parsedText[index].lower()
        if parsedText[index] in rowVector:
            rowVector[parsedText[index]] +=1
        else:
            rowVector[parsedText[index]] = 1
    
    if score >= 4:
        rowVector["<label>"] = "good"
    else:
        rowVector["<label>"] = "bad"
        
    unigramVM.append(rowVector)
    

data.apply(getUnigramReviewVectors, axis = 1)
    
    
    

# SAVE UVM

In [ ]:
#pd.DataFrame(unigramVM).to_csv("UnigramVectorModel.csv", index = False)

# GET TFIDF UNIGRAM MODEL

In [ ]:
#make inverted index
invertedIndex = {}
'''
invertedIndex
the => {index : 1}
'''

def makeInvertedIndex(row):
    #get the bigrams
    #for each bigram -> add a count to the inverted index
    #do not double count
    
    #get bigrams
    text = row["text"]
    parsedText = re.findall("(\w+)", text)
    
    for index in range(len(parsedText)):
        
        parsedText[index] = stemmer.stem(parsedText[index])
        parsedText[index] = parsedText[index].lower()
        eachWord = parsedText[index]
    
        if eachWord not in invertedIndex:
            invertedIndex[eachWord] = [1, {}]

        if row.name not in invertedIndex[eachWord]:
            invertedIndex[eachWord][0] += 1
            invertedIndex[eachWord][1][row.name] = 1

In [ ]:
data.apply(makeInvertedIndex, axis = 1)

In [ ]:
TFIDFVM = []
def getTFIDFReviewVectors(row):
    
    #we have data,unigrams,bigrams
    rowVector = {}
    text = row["text"]
    score = row["score"]
    parsedText = re.findall("(\w+)", text)
    
    for index in range(len(parsedText)):
        parsedText[index] = stemmer.stem(parsedText[index])
        parsedText[index] = parsedText[index].lower()
        
        if parsedText[index] in rowVector:
            rowVector[parsedText[index]] +=1
        else:
            rowVector[parsedText[index]] = 1
        
    #get individual counts
    for eachWord in rowVector:
        
        TF = rowVector[eachWord]/unigramDict[eachWord]
        
        term = len(invertedIndex[eachWord][1])
        
        
        if term == 0:
            term = 1
            
        IDF = math.log2(2000/term)
            
        if IDF < 0:
            print(eachWord)
            
        
    if score >= 4:
        rowVector["<label>"] = "good"
    else:
        rowVector["<label>"] = "bad"
        
    TFIDFVM.append(rowVector)
    



In [ ]:

data.apply(getTFIDFReviewVectors, axis = 1)

In [ ]:
#pd.DataFrame(TFIDFVM).to_csv("UniTFIDFModel.csv", index = False)